# UNET Class and other pre-requisites to load the model here

In [1]:
import torch
import torch.nn as nn #imports necessary neural network toolbox
import torch.nn.functional as nnfunc 

class DoubleConv(nn.Module):
    """
    Double Convolution Block - Basic building block of U-Net
    Pattern: Conv -> BatchNorm -> ReLU -> Conv -> BatchNorm -> ReLU
    """
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),#the first convolution of the double conv.
            nn.BatchNorm2d(out_channels),
            # batch norm will help normalise the effects of activation functions
            # so that the model will be more stable and fast as per Geek4Geeks.
            nn.ReLU(inplace=True), # <- the activation function
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1), #second convolution
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, tensor):
        """
        This confused me initially, I thought that the code under def__init__
        would be enough, but according to Geeks4Geeks: 
        https://www.geeksforgeeks.org/deep-learning/understanding-the-forward-function-output-in-pytorch/
        
        The forward method is PyTorch's implementation of a forward pass:
        that's when the data/tensor is passed through the model
        """
        return self.double_conv(tensor)

class UNET(nn.Module):
    """
    U-Net Architecture for Medical Image Segmentation
    encoder (Downsampling):  Input -> Conv -> Pool -> Conv -> Pool -> bottleneck
    decoder (Upsampling):    bottleneck -> Upsample -> Concat -> Conv -> ... -> Output
    skip connections: Connect encoder features directly to decoder for fine details
    """
    def __init__(self, in_channels, out_channels):
        super(UNET, self).__init__() 
        """
        A super().__init__() is necessary to initialise the base nn.Module 
        so that dependencies and everything works for neural networks in PyTorch
        
        Args:
            in_channels: number of input channels, 1 for grayscale
            out_channels: number of output channels, 1 for binary segmentation
            
        The U-Net paper has the following encoder implementation:
        1) Double conv (64 channels)
        2) Max pool 2x2 + Double conv (128 chnnels)  
        3) Max pool 2x2 + Double conv (256 chanels)
        4) Max pool 2x2 + Double conv (512 channls)
        5) Max pool 2x2 + Double conv (1024 chanels)
        
        Using this, there are 5 different double conv operations
        but we reuse the same max pool operation because the double conv
        has to account for changes in channel dimensions
        """
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)  # halves spatial dimensions
        
        # encoder path (Contracting/Downsampling)
        # Each step: DoubleConv -> MaxPool (except bottleneck)
        self.dconv1 = DoubleConv(in_channels, 64)   # Input -> 64 channes   (128x128 -> 128x128)
        self.dconv2 = DoubleConv(64, 128)           # 64 -> 128 channels     (64x64 -> 64x64)
        self.dconv3 = DoubleConv(128, 256)          # 128 -> 256 chanels    (32x32 -> 32x32)
        self.dconv4 = DoubleConv(256, 512)          # 256 -> 512 chanels    (16x16 -> 16x16)
        self.dconv5 = DoubleConv(512, 1024)         # 512 -> 1024 chanels   (8x8 -> 8x8) bottleneck
        
        # decoder path (Expanding/Upsampling)
        # these are the variables for the decoder - upsampling is the opposite of max pooling
        
        # Level 4: from bottleneck (1024 ch, 8x8) -> Level 4 encoder (512 ch, 16x16)
        self.upsample4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)  # 8x8 -> 16x16
        self.upconv4 = DoubleConv(1024, 512)  # 1024 channels (512+512 from skip) -> 512
        
        # Level 3: from level 4 (512 ch, 16x16) -> Level 3 encoder (256 ch, 32x32)  
        self.upsample3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)   # 16x16 -> 32x32
        self.upconv3 = DoubleConv(512, 256)   # 512 channels (256+256 from skip) -> 256
        
        # Level 2: from level 3 (256 ch, 32x32) -> Level 2 encoder (128 ch, 64x64)
        self.upsample2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)   # 32x32 -> 64x64
        self.upconv2 = DoubleConv(256, 128)   # 256 channels (128+128 from skip) -> 128
        
        # Level 1: from level 2 (128 ch, 64x64) -> Level 1 encoder (64 ch, 128x128)
        self.upsample1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)    # 64x64 -> 128x128
        self.upconv1 = DoubleConv(128, 64)    # 128 channels (64+64 from skip) -> 64
        
        # OUTPUT LAYER
        # after this the last step is one final 1x1 conv layer
        self.outconv = nn.Conv2d(64, 1, kernel_size=1)  # 64 -> 1 channel (segmentation mask)
    
    def _concat(self, up_tensor, skip_tensor):
        """
        Concatenate upsampled tensor with skip connection from encoder.
        handle size mismatches by padding the smaller tensor.
        
        Skip connections are crucial because they:
        1. Preserve fine spatial details lost during downsampling
        2. Help gradients flow better during backpropagation  
        3. Combine low-level features (edges) with high-level features (semantics)
        """
        # Check if spatial dimensions match between upsampled and skip tensors
        if up_tensor.size(2) != skip_tensor.size(2) or up_tensor.size(3) != skip_tensor.size(3):
            # Calculate differences in Height and Width
            y_ax_diff = skip_tensor.size(2) - up_tensor.size(2)  # height difference
            x_ax_diff = skip_tensor.size(3) - up_tensor.size(3)  # width difference

            # Pad order in F.pad = [left, right, top, bottom]
            # Distribute padding evenly, with extra pixels on right/bottom if odd
            up_tensor = nnfunc.pad(
                up_tensor,
                [x_ax_diff // 2, x_ax_diff - x_ax_diff // 2,   # [left, right]
                 y_ax_diff // 2, y_ax_diff - y_ax_diff // 2]   # [top, bottom]
            )
        
        # concatenate along channel dimension (dim=1)
        # noting that tensor format: [batch, channel, height, width]
        # result: channels = skip_channels + up_channels
        return torch.cat([skip_tensor, up_tensor], dim=1)
    
    def forward(self, tensor):
        """
        Forward pass through U-Net
        
        Data Flow:
        Input -> encoder (extract features) -> bottleneck -> decoder (reconstruct) -> Output
        
        Skip connections preserve spatial information lost during downsampling
        
        input tensor shape: (batch_size, channels, height, width)
        echocardiograms frames: (batch_size, 1, 112, 112)
        """
        
        # encoder path (steps 1-9) - Extract features at multiple scales
        # save encoder outputs for skip connections (step1, step3, step5, step7)
        
        
        step1 = self.dconv1(tensor)           # Input(B,1,112,112) -> (B,64,112,112)
        step2 = self.max_pool(step1)          # (B,64,112,112) -> (B,64,56,56)
        
        step3 = self.dconv2(step2)            # (B,64,56,56) -> (B,128,56,56)  
        step4 = self.max_pool(step3)          # (B,128,56,56) -> (B,128,28,28)
        
        step5 = self.dconv3(step4)            # (B,128,28,28) -> (B,256,28,28)
        step6 = self.max_pool(step5)          # (B,256,28,28) -> (B,256,14,14)
        
        step7 = self.dconv4(step6)            # (B,256,14,14) -> (B,512,14,14)
        step8 = self.max_pool(step7)          # (B,512,14,14) -> (B,512,7,7)
        
        step9 = self.dconv5(step8)            # (B,512,7,7) -> (B,1024,7,7) bottleneck
        
        # decoder PATH - Reconstruct spatial resolution with skip connections
        # Each step: Upsample -> Concatenate with encoder -> Process with DoubleConv
        
        # Level 4: Combine bottleneck with encoder level 4 (step7)
        step10 = self.upsample4(step9)        # (B,1024,7,7) -> (B,512,14,14)
        step11 = self._concat(step10, step7)  # Concat: (B,512,14,14) + (B,512,14,14) -> (B,1024,14,14)
        step12 = self.upconv4(step11)         # (B,1024,14,14) -> (B,512,14,14)

        # Level 3: Combine with encoder level 3 (step5) 
        step13 = self.upsample3(step12)       # (B,512,14,14) -> (B,256,28,28)
        step14 = self._concat(step13, step5)  # Concat: (B,256,28,28) + (B,256,28,28) -> (B,512,28,28)
        step15 = self.upconv3(step14)         # (B,512,28,28) -> (B,256,28,28)

        # Level 2: Combine with encoder level 2 (step3)
        step16 = self.upsample2(step15)       # (B,256,28,28) -> (B,128,56,56)
        step17 = self._concat(step16, step3)  # Concat: (B,128,56,56) + (B,128,56,56) -> (B,256,56,56)
        step18 = self.upconv2(step17)         # (B,256,56,56) -> (B,128,56,56)

        # Level 1: Final upsampling to original resolution, combine with step1
        step19 = self.upsample1(step18)       # (B,128,56,56) -> (B,64,112,112)
        step20 = self._concat(step19, step1)  # Concat: (B,64,112,112) + (B,64,112,112) -> (B,128,112,112)
        step21 = self.upconv1(step20)         # (B,128,112,112) -> (B,64,112,112)

        # output - Map feature channels to segmentation mask
        # Returns raw logits (before sigmoid) - loss function will handle this
        return self.outconv(step21)           # (B,64,112,112) -> (B,1,112,112)

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def dice_coeff(predictions, targets, smooth=1e-6):
    """
    no dice loss function in pytorch
    args:
        predictions: Sigmoid probabilities [0,1] 
        targets: Binary ground truth [0,1]
        smooth: Smoothing factor to avoid division by zero
    returns:
        dice_score: Float between 0 and 1
    """
    # Flatten tensors for easier calculation
    pred_flat = predictions.reshape(-1)
    target_flat = targets.reshape(-1)
    
    # Calculate intersection (overlap)
    intersection = (pred_flat * target_flat).sum()
    
    # Calculate union (total covered area)
    union = pred_flat.sum() + target_flat.sum()
    
    # Dice coefficient: 2 * intersection / union
    dice = (2.0 * intersection + smooth) / (union + smooth)
    
    return dice


def CombinedBCEDiceLoss(predictions, targets, bce_weight=0.5, dice_weight=0.5):
    '''
    Combined BCE + Dice Loss for binary segmentation
    args:
        1.predictions (torch.Tensor): the raww model outputs (logits) from U-Net
            - expecting shape: (batch_size, 1, height, width) 
            
        2.targets (torch.Tensor): the ground truth binary masks
            - shape: (batch_size, 1, height, width) - same as predictions
            
        3. bce_weight (float):weighting for the bce component, set to 0.5 for equal weight with dice
            - BCE is good at: sharp boundaries, overall pixel classification
            
        4. dice_weight (float): weighting for the dice component, set to 0.5
            - Dice is good at: handling class imbalance, shape preservation
    
    returns:
        tuple: (combined_loss, bce_component, dice_component)
            - combined_loss (torch.Tensor): Weighted sum for backpropagation
            - bce_component (float): BCE loss value for monitoring
            - dice_component (float): Dice loss value for monitoring
    '''
    # BCE part
    bce_loss = nn.BCEWithLogitsLoss()(predictions, targets)
    
    # Dice part
    predictions_sigmoid = torch.sigmoid(predictions)  #using sigmoid function to perform binary classification of the pixels
    dice_score = dice_coeff(predictions_sigmoid, targets) #compares the predictions agains the ground truth
    dice_loss = 1.0 - dice_score
    
    # Combined loss
    combined_loss = (bce_weight * bce_loss) + (dice_weight * dice_loss)
    
    return combined_loss, bce_loss.item(), dice_loss.item()


In [3]:
# Set device for computation (GPU if available, else CPU)
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Model First:
def load_trained_model(path="best_unet_adamw_cosine.pt"):  # Fixed typo in filename
    """Load the trained U-Net model from checkpoint"""
    model = UNET(1, 1).to(device)
    
    if os.path.exists(path):  # Check if model exists
        try:
            checkpoint = torch.load(path, map_location=device, weights_only=False)
            model.load_state_dict(checkpoint['model_state_dict'])
            print("Model loaded")
            model.eval()
            return model, checkpoint
        except Exception as e:
            print("Model not loaded")
            return None, None
    else:
        print("Error")
        return None, None




In [4]:
# Make sure to load the trained model first -- see code above
# Train on the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using: {device}")

from pathlib import Path

def predict_one_folder(model, folder, threshold=0.5):
    '''
    Use this to predict 1 folder first to see the results, and prototype the video output

    args: 
        model: UNET(in_chan, out_chan)
        folder: the path to the folder for prediction-- use test folder instead of train/val
        threshold: threshold for binary pred set to 0.5

    returns: a dictionary called predictions
        frame: framename relating to the predicted frame
        frame_image: the original frame image
        mask: binary mask as numpy arrays (just like how original segmentation mask was created)
        pixel_count: pixel count for the mask
    '''
    folder = Path(folder)
    frame_paths = sorted(folder.glob("*.png")) # look for .png files in the specified folder

    predictions = {
        'frame': [],           # framename 
        'frame_image': [],     # frameimage
        'mask': [],            # binary mask
        'pixel_count': []      # pixel count for the mask
    }
    model.eval()

    with torch.no_grad(): # disable backprop
        for frame_path in frame_paths:
            try:
                # loading the original frame
                frame = cv2.imread(str(frame_path), cv2.IMREAD_GRAYSCALE) # specifying that it's grayscale

                # Normalise to [0,1] for gradient stability, and to help the sigmoid act function to work with the threshold of 0.5
                frame_normalised = frame.astype(np.float32)/255.0
                frame_tensor = torch.from_numpy(frame_normalised) # creates tensor shape (112, 112) corresp to H*W
                frame_tensor = frame_tensor.unsqueeze(0) # add channel dim, = 1 so shape is (1,112,112)
                frame_tensor = frame_tensor.unsqueeze(0) # add batch dim, so shape is (1,1,112,112)
                frame_tensor = frame_tensor.to(device) # use GPU

                # run the model to output logits which are just raw numbers
                with torch.amp.autocast(device_type=device.type, enabled=(device.type=="cuda")):
                    logits = model(frame_tensor) # creates raw logits
                    probs = torch.sigmoid(logits) # after sigmoid it changes this to [0,1]
                    binary_mask = (probs > threshold).float() # apply the 0.5 threshold

                mask_np = binary_mask.squeeze().cpu().numpy()  # Move to CPU first, then convert
                pixel_count = int(np.sum(mask_np))

                # store the result of each frame in frame_path in the dict.
                predictions['frame'].append(frame_path.name)         # filename
                predictions['frame_image'].append(frame)             # original image (112x112)
                predictions['mask'].append(mask_np)                  # binary mask (112x112)
                predictions['pixel_count'].append(pixel_count)       # pix count for later

            except Exception:
                print("error")
                break

    return predictions

Using: cuda


# Code to Calculate LVEF per video here

In [ ]:
# Add missing imports
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import glob
import torch
import cv2
import numpy as np
import os

# 1. First, load your trained model
loaded_model, checkpoint_info = load_trained_model("best_unet_adamw_cosine.pt")

def predict_one_fast(model, folder, threshold=0.5, batch_size=64):
    """
    Fast batch processing without DataLoader workers (Windows-friendly)
    """
    folder = Path(folder)
    frame_paths = sorted(folder.glob("*.png"))
    
    if not frame_paths:
        return {'frame': [], 'frame_image': [], 'mask': [], 'pixel_count': []}
    
    predictions = {
        'frame': [],
        'frame_image': [],
        'mask': [],
        'pixel_count': []
    }
    
    model.eval()
    with torch.no_grad():
        # Process in batches manually (no DataLoader)
        for i in range(0, len(frame_paths), batch_size):
            batch_paths = frame_paths[i:i+batch_size]
            batch_tensors = []
            batch_frames = []
            batch_names = []
            
            # Load batch of frames
            for frame_path in batch_paths:
                try:
                    frame = cv2.imread(str(frame_path), cv2.IMREAD_GRAYSCALE)
                    if frame is None:
                        continue
                        
                    frame_normalised = frame.astype(np.float32) / 255.0
                    frame_tensor = torch.from_numpy(frame_normalised).unsqueeze(0)  # Add channel dim
                    
                    batch_tensors.append(frame_tensor)
                    batch_frames.append(frame)
                    batch_names.append(frame_path.name)
                except Exception as e:
                    print(f"Error loading frame {frame_path}: {e}")
                    continue
            
            if not batch_tensors:
                continue
                
            # Stack tensors and move to device
            batch_tensor = torch.stack(batch_tensors).to(device)
            
            # Process batch through model
            try:
                with torch.amp.autocast(device_type=device.type, enabled=(device.type=="cuda")):
                    logits = model(batch_tensor)
                    probs = torch.sigmoid(logits)
                    binary_masks = (probs > threshold).float()
                
                # Move results back to CPU
                binary_masks_cpu = binary_masks.squeeze(1).cpu().numpy()
                
                # Store results
                for mask_np, frame_name, frame in zip(binary_masks_cpu, batch_names, batch_frames):
                    pixel_count = int(np.sum(mask_np))
                    
                    predictions['frame'].append(frame_name)
                    predictions['frame_image'].append(frame)
                    predictions['mask'].append(mask_np)
                    predictions['pixel_count'].append(pixel_count)
                    
            except Exception as e:
                print(f"Error processing batch: {e}")
                continue
    
    return predictions

def calculate_lvef_from_predictions(predictions):
    """Calculate LVEF from pixel counts"""
    if not predictions or len(predictions['pixel_count']) < 2:
        return None
    
    pixel_counts = np.array(predictions['pixel_count'])
    
    esv_pixel = np.min(pixel_counts)
    esv_frame_index = np.argmin(pixel_counts)
    edv_pixel = np.max(pixel_counts)
    edv_frame_index = np.argmax(pixel_counts)
    
    if edv_pixel > 0:
        lvef_pixel = ((edv_pixel - esv_pixel) / edv_pixel) * 100
    else:
        lvef_pixel = 0
    
    return {
        'ESV_pixel': int(esv_pixel),
        'EDV_pixel': int(edv_pixel),
        'LVEF_pixel': round(lvef_pixel, 2),
        'ESV_frame': predictions['frame'][esv_frame_index],
        'EDV_frame': predictions['frame'][edv_frame_index],
        'total_frames': len(predictions['frame'])
    }

def process_multiple_videos_fast(video_directories, model, batch_size=16):
    """
    Process multiple videos efficiently without DataLoader workers
    """
    results = []
    
    for video_dir in tqdm(video_directories, desc="Processing videos"):
        try:
            # Use fast batch processing
            predictions = predict_one_fast(model, video_dir, threshold=0.5, batch_size=batch_size)
            
            if not predictions or len(predictions['pixel_count']) == 0:
                continue
            
            lvef_results = calculate_lvef_from_predictions(predictions)
            if lvef_results is None:
                continue
            
            video_id = Path(video_dir).name
            result_row = {
                'Key': video_id,
                'Min_pixel': lvef_results['ESV_pixel'],
                'Max_pixel': lvef_results['EDV_pixel'],
                'EF_pixel': lvef_results['LVEF_pixel'],
                'ESV_frame': lvef_results['ESV_frame'],
                'EDV_frame': lvef_results['EDV_frame'],
                'total_frames': lvef_results['total_frames']
            }
            results.append(result_row)
            
        except Exception as e:
            print(f"Error processing {video_dir}: {e}")
            continue
    
    return pd.DataFrame(results)

def load_ground_truth_data(csv_path="FileList.csv"):
    try:
        gt_df = pd.read_csv(csv_path)
        gt_df['Key'] = gt_df['FileName'].str.replace('.avi', '').str.replace('.mp4', '')
        return gt_df[['Key', 'EF', 'ESV', 'EDV']].copy()
    except Exception as e:
        print(f"Error loading ground truth: {e}")
        return None

def compare_with_ground_truth(predictions_df, ground_truth_df):
    comparison_df = predictions_df.merge(ground_truth_df, on='Key', how='inner')
    comparison_df['EF_error'] = comparison_df['EF_pixel'] - comparison_df['EF']
    comparison_df['EF_error_abs'] = comparison_df['EF_error'].abs()
    return comparison_df

def quick_plot_results(comparison_df):
    """Simplified plotting"""
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Scatter plot
    axes[0].scatter(comparison_df['EF'], comparison_df['EF_pixel'], alpha=0.6)
    axes[0].plot([0, 100], [0, 100], 'r--', label='Perfect Agreement')
    axes[0].set_xlabel('Ground Truth EF (%)')
    axes[0].set_ylabel('Predicted EF (%)')
    axes[0].set_title('Predicted vs Ground Truth LVEF')
    axes[0].legend()
    
    # Error histogram
    axes[1].hist(comparison_df['EF_error'], bins=15, alpha=0.7)
    axes[1].axvline(0, color='red', linestyle='--')
    axes[1].set_xlabel('EF Error (%)')
    axes[1].set_ylabel('Count')
    axes[1].set_title('Error Distribution')
    
    plt.tight_layout()
    plt.show()
    
    # Calculate metrics
    mae = comparison_df['EF_error_abs'].mean()
    rmse = np.sqrt((comparison_df['EF_error']**2).mean())
    correlation = comparison_df['EF'].corr(comparison_df['EF_pixel'])
    
    print(f"📊 Results: MAE={mae:.2f}%, RMSE={rmse:.2f}%, Correlation={correlation:.3f}")
    return mae, rmse, correlation

# 🚀 MAIN EXECUTION (FIXED VERSION)
# 🚀 MAIN EXECUTION (PROCESS ALL VIDEOS)
if loaded_model is not None:
    print("✅ Model loaded successfully!")
    
    # Get video directories
    video_base_path = "EchoNet-Dynamic/frame_outputs"
    test_video_dirs = [d for d in glob.glob(f"{video_base_path}/*") if os.path.isdir(d)]
    
    if test_video_dirs:
        print(f"Found {len(test_video_dirs)} video directories")
        
        # Process ALL videos (removed the [:20] slice)
        print(f"Processing ALL {len(test_video_dirs)} videos...")
        
        # Use fast processing (no DataLoader workers)
        predictions_df = process_multiple_videos_fast(test_video_dirs, loaded_model, batch_size=8)
        
        if len(predictions_df) > 0:
            print(f"\n✅ Processed {len(predictions_df)} videos")
            print("\nSample results:")
            print(predictions_df.head())
            
            # Quick statistics
            print(f"\n📊 Quick Stats:")
            print(f"   Average LVEF: {predictions_df['EF_pixel'].mean():.1f}%")
            print(f"   LVEF Range: {predictions_df['EF_pixel'].min():.1f}% - {predictions_df['EF_pixel'].max():.1f}%")
            
            # Save predictions
            predictions_df.to_csv("lvef_predictions_all_videos.csv", index=False)
            print("💾 Results saved to lvef_predictions_all_videos.csv")
            
            # Load ground truth and compare
            ground_truth_df = load_ground_truth_data("FileList.csv")
            if ground_truth_df is not None:
                comparison_df = compare_with_ground_truth(predictions_df, ground_truth_df)
                if len(comparison_df) > 0:
                    print(f"\n✅ Matched {len(comparison_df)} videos with ground truth")
                    
                    # Quick visualization
                    mae, rmse, correlation = quick_plot_results(comparison_df)
                    
                    # Save comparison results
                    comparison_df.to_csv("lvef_comparison_all_videos.csv", index=False)
                    print("💾 Comparison results saved to lvef_comparison_all_videos.csv")
                else:
                    print("❌ No matching videos found with ground truth")
        else:
            print("❌ No videos processed")
    else:
        print("❌ No video directories found")
        print("Please check if EchoNet-Dynamic/frame_outputs/ exists")
else:
    print("❌ Model not loaded")
# Quick test function
def test_single_video():
    """Test on one video"""
    if loaded_model is not None:
        video_dirs = [d for d in glob.glob("EchoNet-Dynamic/frame_outputs/*") if os.path.isdir(d)]
        if video_dirs:
            test_dir = video_dirs[0]
            print(f"\n🔍 Testing: {Path(test_dir).name}")
            
            predictions = predict_one_fast(loaded_model, test_dir, batch_size=8)
            if predictions:
                lvef_results = calculate_lvef_from_predictions(predictions)
                if lvef_results:
                    print(f"   Frames: {lvef_results['total_frames']}")
                    print(f"   ESV: {lvef_results['ESV_pixel']} pixels")
                    print(f"   EDV: {lvef_results['EDV_pixel']} pixels")
                    print(f"   LVEF: {lvef_results['LVEF_pixel']:.2f}%")

# Uncomment to test single video first
# test_single_video()

Model loaded
✅ Model loaded successfully!
Found 10030 video directories
Processing ALL 10030 videos...


Processing videos:  38%|███▊      | 3787/10030 [2:24:57<3:21:32,  1.94s/it] 